In [175]:
# impoting libraries import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

In [176]:
#  Loading the data and understanding
df = pd.read_csv("C:\\Users\\DELL\\Desktop\\Assignments\\Recommendation System\\anime.csv")

In [177]:
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [178]:
df.shape

(12294, 7)

In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [180]:
df.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [181]:
# Handling missing values 
from sklearn.impute import KNNImputer

In [182]:
impute = KNNImputer()
impute = KNNImputer(n_neighbors=5)

In [183]:
df[['rating']] = impute.fit_transform(df[['rating']])

In [184]:
# Dropping null values from genre and type columns 
df = df.dropna(subset=['genre', 'type'])

In [185]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12210 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12210 non-null  int64  
 1   name      12210 non-null  object 
 2   genre     12210 non-null  object 
 3   type      12210 non-null  object 
 4   episodes  12210 non-null  object 
 5   rating    12210 non-null  float64
 6   members   12210 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 763.1+ KB


In [186]:
#  Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [187]:
# Create pivot table (anime name x type) from training data
item_matrix = train_df.pivot_table(index='name', columns='type', values='rating')
item_matrix = item_matrix.fillna(0)

In [188]:
# Normalize & compute cosine similarity
scaler = StandardScaler()
item_matrix_scaled = scaler.fit_transform(item_matrix)

item_sim_matrix = cosine_similarity(item_matrix_scaled)
item_sim_df = pd.DataFrame(item_sim_matrix, index=item_matrix.index, columns=item_matrix.index)

In [189]:
# Recommendation Function
def recommend_similar_anime(target_anime, top_n=5):
    if target_anime not in item_sim_df.columns:
        return []
    
    similar_scores = item_sim_df[target_anime].sort_values(ascending=False)
    recommendations = similar_scores.drop(target_anime).head(top_n).index.tolist()
    return recommendations

In [190]:
#  Example recommendation
print("Recommended animes similar to 'One Piece':\n")
print(recommend_similar_anime("One Piece", top_n=5))

Recommended animes similar to 'One Piece':

['Kuroko no Basket 2nd Season', 'Uchuu Kyoudai', 'Kiseijuu: Sei no Kakuritsu', 'Kingdom 2nd Season', 'JoJo no Kimyou na Bouken: Diamond wa Kudakenai']


In [191]:
# Evaluation (genre-based relevance)F
y_true = []
y_pred = []

for _, row in test_df.iterrows():
    query_anime = row['name']
    true_genres = set(row['genre'].split(', '))

    recs = recommend_similar_anime(query_anime, top_n=5)
    if not recs:
        continue  # skip if the anime isn't in training data

    match = 0
    for rec_name in recs:
        rec_row = train_df[train_df['name'] == rec_name]
        if rec_row.empty:
            continue
        rec_genres = set(rec_row.iloc[0]['genre'].split(', '))
        if true_genres & rec_genres:
            match += 1

    y_true.append(1)
    y_pred.append(1 if match > 0 else 0)




In [192]:
# Print Evaluation Metrics
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print("\n📊 Evaluation Results:")
print(f"Precision: {precision:.5f}")
print(f"Recall:    {recall:.5f}")
print(f"F1-Score:  {f1:.5f}")


📊 Evaluation Results:
Precision: 0.00000
Recall:    0.00000
F1-Score:  0.00000


C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Interveiw Q&A